# End-to-End Remote Cardiac Monitoring Integration using HL7 (v2.x)

**What this is:** A mini project, end-to-end HL7 v2 workflow that simulates how a hospital registers a patient, places a remote cardiac monitoring order (Holter), and receives structured results from a HealthCast-like monitoring platform.

**Systems in this story**
- **HIS**: Hospital Information System
- **CIS**: Cardiology Information System
- **Remote Monitoring Platform**: HealthCast-like platform
- **Integration Layer**: This notebook (routes/prints messages)

---

## S — Situation
A patient (Juana López) attends an outpatient cardiology visit. Multiple systems must stay synchronized so orders and results are reliably linked to the right patient and encounter.

## T — Task
Ensure a clean flow of:
1) patient identity + visit context  
2) a cardiac monitoring order  
3) structured monitoring results

## A — Action (HL7 messages)
We use HL7 v2 messages:
- **ADT^A01** (patient registration / visit creation)
- **ORM^O01** (order for Holter)
- **ORU^R01** (results)

---

# 0) Setup


In [2]:
from datetime import datetime
from uuid import uuid4

HL7_VERSION = "2.5"

def hl7_ts(dt: datetime | None = None) -> str:
    """HL7 TS format: YYYYMMDDHHMMSS"""
    dt = dt or datetime.now()
    return dt.strftime("%Y%m%d%H%M%S")

def new_msg_id(prefix="MSG") -> str:
    return f"{prefix}{uuid4().hex[:8].upper()}"

def build_msh(
    sending_app: str,
    sending_fac: str,
    receiving_app: str,
    receiving_fac: str,
    msg_type: str,      # e.g., "ADT^A01"
    msg_id: str,
    dt: datetime | None = None,
    processing_id: str = "P",
    version: str = HL7_VERSION,
) -> str:
    # MSH-1 is field separator '|' and MSH-2 is encoding chars '^~\&'
    return "|".join([
        "MSH",
        "^~\\&",
        sending_app,
        sending_fac,
        receiving_app,
        receiving_fac,
        hl7_ts(dt),
        "",                 # MSH-8 Security (unused)
        msg_type,           # MSH-9 Message Type
        msg_id,             # MSH-10 Message Control ID
        processing_id,      # MSH-11 Processing ID
        version,            # MSH-12 Version ID
    ])

def segment(*fields: str) -> str:
    return "|".join(fields)

def hl7_message(*segments: str) -> str:
    # HL7 typically uses \r as segment terminator; printing with \n is easier to read.
    return "\r".join(segments) + "\r"

def pretty_hl7(msg: str) -> str:
    # Render with newlines for readability in notebooks
    return msg.replace("\r", "\n").strip()

def print_hl7(title: str, msg: str):
    print(f"\n{'='*80}\n{title}\n{'='*80}")
    print(pretty_hl7(msg))


# 1) Locked Clinical Context (the “real context”)
We will follow one outpatient cardiology workflow:

- **Patient**: Juana López (F, 1988-04-12), Patient ID: HOSP12345  
- **Visit**: Outpatient cardiology  
- **Reason**: Palpitations and dizziness  
- **Order**: 24-hour Ambulatory ECG (Holter)  
- **Results** (example):
  - Duration: 24 hours
  - Average HR: 72 bpm
  - Min HR: 55 bpm
  - Max HR: 130 bpm
  - Rhythm: Sinus rhythm
  - Arrhythmias: None detected

---

# 2) Architecture Overview
```text
[ HIS ]  --(ADT^A01)-->  [ Integration Layer ]  --(ORM^O01)-->  [ Remote Monitoring Platform ]
                                                         \
                                                          \--(ORU^R01)-->  [ HIS / CIS ]


In [4]:

# Patient + visit context (locked)
patient = {
    "patient_id": "HOSP12345",
    "last_name": "Lopez",
    "first_name": "Juana",
    "dob": "19880412",     # HL7 date
    "sex": "F",
}

visit = {
    "patient_class": "O",              # Outpatient
    "location": "CARDIOLOGY",          # PV1-3 (simplified)
    "attending_doc_id": "1234",
    "attending_last": "Garcia",
    "attending_first": "Ana",
}


# 3) ACTION 1 — Patient Registration (ADT^A01)
**Meaning:** The HIS establishes patient identity and visit context so downstream systems can safely process orders/results.


In [6]:
def build_adt_a01(
    patient: dict,
    visit: dict,
    sending_app="HIS",
    sending_fac="HOSPITAL_MADRID",
    receiving_app="INTEGRATION",
    receiving_fac="INT_LAYER",
    dt: datetime | None = None,
) -> str:
    msg_id = new_msg_id("ADT")
    msh = build_msh(
        sending_app=sending_app,
        sending_fac=sending_fac,
        receiving_app=receiving_app,
        receiving_fac=receiving_fac,
        msg_type="ADT^A01",
        msg_id=msg_id,
        dt=dt,
    )

    pid = segment(
        "PID",
        "1",
        "",                          # PID-2 (unused)
        patient["patient_id"],       # PID-3 Patient ID
        "",
        f'{patient["last_name"]}^{patient["first_name"]}',  # PID-5 Patient Name
        "",
        patient["dob"],              # PID-7 DOB
        patient["sex"],              # PID-8 Sex
    )

    pv1 = segment(
        "PV1",
        "1",
        visit["patient_class"],      # PV1-2
        visit["location"],           # PV1-3 (simplified)
        "",
        "",
        f'{visit["attending_doc_id"]}^{visit["attending_last"]}^{visit["attending_first"]}',  # PV1-7 (simplified)
    )

    return hl7_message(msh, pid, pv1)

adt_msg = build_adt_a01(patient, visit, dt=datetime(2025, 10, 10, 9, 0, 0))
print_hl7("ADT^A01 — Patient Registration (HIS → Integration Layer)", adt_msg)



ADT^A01 — Patient Registration (HIS → Integration Layer)
MSH|^~\&|HIS|HOSPITAL_MADRID|INTEGRATION|INT_LAYER|20251010090000||ADT^A01|ADTA01013D1|P|2.5
PID|1||HOSP12345||Lopez^Juana||19880412|F
PV1|1|O|CARDIOLOGY|||1234^Garcia^Ana


# 4) ACTION 2 — Cardiac Monitoring Order (ORM^O01)
**Meaning:** The CIS places an order to activate a device workflow (24-hour Holter) in the remote monitoring platform.


In [7]:
def build_orm_o01(
    patient: dict,
    order: dict,
    sending_app="CIS",
    sending_fac="CARDIOLOGY_DEPT",
    receiving_app="INTEGRATION",
    receiving_fac="INT_LAYER",
    dt: datetime | None = None,
) -> str:
    msg_id = new_msg_id("ORM")
    msh = build_msh(
        sending_app=sending_app,
        sending_fac=sending_fac,
        receiving_app=receiving_app,
        receiving_fac=receiving_fac,
        msg_type="ORM^O01",
        msg_id=msg_id,
        dt=dt,
    )

    pid = segment(
        "PID",
        "1",
        "",
        patient["patient_id"],
        "",
        f'{patient["last_name"]}^{patient["first_name"]}',
        "",
        patient["dob"],
        patient["sex"],
    )

    # ORC — Order Control (simplified)
    orc = segment(
        "ORC",
        "NW",                # New Order
        order["placer_order"],# Placer Order Number
    )

    # OBR — Observation Request (simplified)
    obr = segment(
        "OBR",
        "1",
        order["placer_order"],          # OBR-2
        "",
        f'{order["service_id"]}^{order["service_text"]}',  # OBR-4
        "",
        hl7_ts(dt),                     # OBR-7 Observation Date/Time
        "",
        "",
        "",
        "",
        "",
        order["reason"],                # OBR-13 (Reason for Study) simplified
        "",
        f'{order["ordering_doc_id"]}^{order["ordering_last"]}^{order["ordering_first"]}',  # OBR-16 (Ordering Provider)
    )

    return hl7_message(msh, pid, orc, obr)

order = {
    "placer_order": "ORD0001",
    "service_id": "HOLTER24H",
    "service_text": "Ambulatory ECG Monitoring 24h",
    "reason": "Palpitations and dizziness",
    "ordering_doc_id": "5678",
    "ordering_last": "Sanchez",
    "ordering_first": "Lucia",
}

orm_msg = build_orm_o01(patient, order, dt=datetime(2025, 10, 10, 10, 0, 0))
print_hl7("ORM^O01 — Holter Order (CIS → Integration Layer)", orm_msg)



ORM^O01 — Holter Order (CIS → Integration Layer)
MSH|^~\&|CIS|CARDIOLOGY_DEPT|INTEGRATION|INT_LAYER|20251010100000||ORM^O01|ORM8B154E4B|P|2.5
PID|1||HOSP12345||Lopez^Juana||19880412|F
ORC|NW|ORD0001
OBR|1|ORD0001||HOLTER24H^Ambulatory ECG Monitoring 24h||20251010100000||||||Palpitations and dizziness||5678^Sanchez^Lucia


# 5) ACTION 3 — Monitoring Results (ORU^R01)
**Meaning:** The remote monitoring platform sends structured clinical results back to hospital systems.


In [8]:
def build_oru_r01(
    patient: dict,
    order: dict,
    results: dict,
    sending_app="REMOTE_PLATFORM",
    sending_fac="HEALTHCAST_LIKE",
    receiving_app="INTEGRATION",
    receiving_fac="INT_LAYER",
    dt: datetime | None = None,
) -> str:
    msg_id = new_msg_id("ORU")
    msh = build_msh(
        sending_app=sending_app,
        sending_fac=sending_fac,
        receiving_app=receiving_app,
        receiving_fac=receiving_fac,
        msg_type="ORU^R01",
        msg_id=msg_id,
        dt=dt,
    )

    pid = segment(
        "PID",
        "1",
        "",
        patient["patient_id"],
        "",
        f'{patient["last_name"]}^{patient["first_name"]}',
        "",
        patient["dob"],
        patient["sex"],
    )

    # OBR references the original order (simplified)
    obr = segment(
        "OBR",
        "1",
        order["placer_order"],
        "",
        f'{order["service_id"]}^{order["service_text"]}',
        "",
        hl7_ts(dt),
    )

    # OBX segments (structured observations)
    obx_segments = []
    i = 1

    def obx_nm(code, text, value, units, ref=""):
        nonlocal i
        seg = segment(
            "OBX",
            str(i),
            "NM",                    # Numeric
            f"{code}^{text}",
            "",
            str(value),
            units,
            ref,
            "N",                     # Normal flag (simplified)
        )
        i += 1
        return seg

    def obx_tx(code, text, value):
        nonlocal i
        seg = segment(
            "OBX",
            str(i),
            "TX",                    # Text
            f"{code}^{text}",
            "",
            value,
        )
        i += 1
        return seg

    obx_segments.append(obx_nm("DUR", "Monitoring Duration", results["duration_hours"], "h"))
    obx_segments.append(obx_nm("HR_AVG", "Average Heart Rate", results["avg_hr"], "bpm", "60-100"))
    obx_segments.append(obx_nm("HR_MIN", "Minimum Heart Rate", results["min_hr"], "bpm"))
    obx_segments.append(obx_nm("HR_MAX", "Maximum Heart Rate", results["max_hr"], "bpm"))
    obx_segments.append(obx_tx("RHYTHM", "Dominant Rhythm", results["rhythm"]))
    obx_segments.append(obx_tx("ARR", "Arrhythmias Detected", results["arrhythmias"]))

    return hl7_message(msh, pid, obr, *obx_segments)

results = {
    "duration_hours": 24,
    "avg_hr": 72,
    "min_hr": 55,
    "max_hr": 130,
    "rhythm": "Sinus rhythm",
    "arrhythmias": "None detected",
}

oru_msg = build_oru_r01(patient, order, results, dt=datetime(2025, 10, 10, 11, 30, 0))
print_hl7("ORU^R01 — Holter Results (Remote Platform → Integration Layer)", oru_msg)



ORU^R01 — Holter Results (Remote Platform → Integration Layer)
MSH|^~\&|REMOTE_PLATFORM|HEALTHCAST_LIKE|INTEGRATION|INT_LAYER|20251010113000||ORU^R01|ORU8F4DE057|P|2.5
PID|1||HOSP12345||Lopez^Juana||19880412|F
OBR|1|ORD0001||HOLTER24H^Ambulatory ECG Monitoring 24h||20251010113000
OBX|1|NM|DUR^Monitoring Duration||24|h||N
OBX|2|NM|HR_AVG^Average Heart Rate||72|bpm|60-100|N
OBX|3|NM|HR_MIN^Minimum Heart Rate||55|bpm||N
OBX|4|NM|HR_MAX^Maximum Heart Rate||130|bpm||N
OBX|5|TX|RHYTHM^Dominant Rhythm||Sinus rhythm
OBX|6|TX|ARR^Arrhythmias Detected||None detected


# 6) End-to-End Summary
We have demonstrated a clean integration flow:

1) **ADT^A01** establishes patient identity + visit context  
2) **ORM^O01** places a Holter order to the remote monitoring platform  
3) **ORU^R01** returns structured monitoring results using OBX segments  

This mirrors how connected care solutions integrate with hospital systems using HL7 interfaces.


In [9]:
print("END-TO-END FLOW (HAPPY PATH)")
print("- ADT^A01: Patient registered (HIS → Integration)")
print("- ORM^O01: Holter ordered (CIS → Integration)")
print("- ORU^R01: Results returned (Remote Platform → Integration → HIS/CIS)")


END-TO-END FLOW (HAPPY PATH)
- ADT^A01: Patient registered (HIS → Integration)
- ORM^O01: Holter ordered (CIS → Integration)
- ORU^R01: Results returned (Remote Platform → Integration → HIS/CIS)


# (Optional) Export the messages to files
If you want to attach HL7 examples to an email/LinkedIn post or keep them as artifacts.


In [10]:
from pathlib import Path

out_dir = Path("hl7_artifacts")
out_dir.mkdir(exist_ok=True)

(out_dir / "ADT_A01.hl7").write_text(adt_msg)
(out_dir / "ORM_O01.hl7").write_text(orm_msg)
(out_dir / "ORU_R01.hl7").write_text(oru_msg)

print(f"Saved HL7 files to: {out_dir.resolve()}")
for p in sorted(out_dir.glob("*.hl7")):
    print("-", p.name)


Saved HL7 files to: /content/hl7_artifacts
- ADT_A01.hl7
- ORM_O01.hl7
- ORU_R01.hl7
